In [1]:
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, LSTM, Bidirectional, Dropout

In [3]:
vocab_size = 12000
embedding_dim = 64
max_length = 1000
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

In [4]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json

--2020-12-27 03:07:15--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.12.240, 172.217.164.176, 172.217.2.112, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.12.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.02s   

2020-12-27 03:07:15 (234 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [5]:
with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])


import numpy as np
labels = np.array(labels)

In [6]:
# Chia features tập train
training_sentences = sentences[0:training_size]
# FIXME: Chia features tập test
test_sentences = sentences[training_size:]
# FIXME: Chia labels tập train
train_labels = labels[0:training_size]
# FIXME: Chia labels tập test
test_labels = labels[training_size:]

In [ ]:
# FIXME: Tạo tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)

# FIXME: Đưa từ vào tokenizer để tạo từ điển

tokenizer.fit_on_texts(training_sentences)

# FIXME: Biến các câu train thành sequences

training_sequences = tokenizer.texts_to_sequences(training_sentences)

# FIXME: Padding cho các train sequences

training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# FIXME: Biến các câu test thành sequences và padding tương ứng

test_sequences = tokenizer.texts_to_sequences(test_sentences)

test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [7]:
# Xây dựng model

# FIXME: Tạo model Sequential

model = Sequential()

# FIXME: Thêm một lớp Embedding

model.add(Embedding(vocab_size, embedding_dim, input_length= max_length))

#Fix

#model.add(Bidirectional(LSTM(64, return_sequences=True)))

#model.add(Bidirectional(LSTM(32, return_sequences=True)))

model.add(GlobalAveragePooling1D())

# FIXME: Thêm một lớp Dense, chú ý có activation

model.add(Dense(10, activation='relu'))

#model.add(Dropout(0.5))
# FIXME: Thêm một lớp Dense cuối, chú ý có activation

model.add(Dense(1, activation='sigmoid'))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 64)          768000    
Total params: 768,000
Trainable params: 768,000
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
model.fit(training_padded, train_labels, epochs=30, validation_data=(test_padded, test_labels))

Epoch 1/30
625/625 [==============================] - 88s 140ms/step - loss: 0.5444 - acc: 0.7207 - val_loss: 0.4471 - val_acc: 0.8208
Epoch 2/30
625/625 [==============================] - 85s 136ms/step - loss: 0.3743 - acc: 0.8310 - val_loss: 0.4496 - val_acc: 0.8396
Epoch 3/30
625/625 [==============================] - 85s 135ms/step - loss: 0.3057 - acc: 0.8928 - val_loss: 0.4108 - val_acc: 0.8201
Epoch 4/30
625/625 [==============================] - 85s 136ms/step - loss: 0.2621 - acc: 0.9093 - val_loss: 0.4053 - val_acc: 0.8332
Epoch 5/30
625/625 [==============================] - 85s 136ms/step - loss: 0.1874 - acc: 0.9329 - val_loss: 0.4362 - val_acc: 0.8360
Epoch 6/30
625/625 [==============================] - 85s 135ms/step - loss: 0.1507 - acc: 0.9466 - val_loss: 0.4579 - val_acc: 0.8360
Epoch 7/30
625/625 [==============================] - 85s 136ms/step - loss: 0.1309 - acc: 0.9572 - val_loss: 0.4775 - val_acc: 0.8326
Epoch 8/30
625/625 [==============================] - 8

In [ ]:
model.save('is_sarcastic.h5')